In [ ]:
!python --version

Python 3.6.9


In [ ]:
# mount drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import os
import json
import csv
import pandas as pd

read_folder_path = "/content/drive/MyDrive/Thesis/Attack Step Prediction/Implementation/APTGen_json/"
write_folder_path = "/content/drive/MyDrive/Thesis/Attack Step Prediction/Implementation/sequences.csv"

# Extract sequence of steps from APTGen dataset

In [ ]:
# # get all sequence files
# files = sorted(os.listdir(read_folder_path))

# # append absolute path from root
# files = [read_folder_path + f for f in files]
# print(files)

['/content/drive/MyDrive/Thesis/Attack Step Prediction/Implementation/APTGen_json/001_collection-collection-8-6816285d.json', '/content/drive/MyDrive/Thesis/Attack Step Prediction/Implementation/APTGen_json/002_collection-collection-8-f58b7445.json', '/content/drive/MyDrive/Thesis/Attack Step Prediction/Implementation/APTGen_json/003_collection-defense_evasion-8-546c6ce.json', '/content/drive/MyDrive/Thesis/Attack Step Prediction/Implementation/APTGen_json/004_collection-discovery-8-d1b6635c.json', '/content/drive/MyDrive/Thesis/Attack Step Prediction/Implementation/APTGen_json/005_collection-discovery-8-dc1746da.json', '/content/drive/MyDrive/Thesis/Attack Step Prediction/Implementation/APTGen_json/006_credential_access-collection-8-7cce214e.json', '/content/drive/MyDrive/Thesis/Attack Step Prediction/Implementation/APTGen_json/007_defense_evasion-collection-8-823d086b.json', '/content/drive/MyDrive/Thesis/Attack Step Prediction/Implementation/APTGen_json/008_defense_evasion-defense_e

In [ ]:
# # extract tactic, tactic id, techniqe, technique id from an attack sequence
# def extract_steps(filename):
#     with open(filename, "r") as sequence_file:
#         sequence = json.load(sequence_file)['scenario']
#         steps = []
#         for step_info in sequence:
#             steps.append(f"{step_info['tactic']} {step_info['tactic_id']} {step_info['technique']} {step_info['technique_id']}")
        
#         return steps

In [ ]:
# # create the csv file containing all the sequence of attack steps
# def create_sequence_of_steps(filename):
#     with open(filename, "w") as write_file:
#         writer = csv.writer(write_file)

#         for file in files:
#             steps = extract_steps(file)        
#             writer.writerow(steps)

#     return filename

In [ ]:
# sequences_csv = create_sequence_of_steps(write_folder_path)

In [ ]:
sequences_csv = "/content/drive/MyDrive/Thesis/Attack Step Prediction/Implementation/sequences.csv"

In [ ]:
def get_data_text():
    sequences_csv = "/content/drive/MyDrive/Thesis/Attack Step Prediction/Implementation/sequences.csv"
    data_text = []
    with open(sequences_csv, "r", newline="") as read_file:
        sequence_reader = csv.reader(read_file)
        for sequence in sequence_reader:
            data_text.append(sequence)
    return data_text

In [ ]:
data_text = get_data_text()
print(data_text[0])
len(data_text)

In [ ]:
def sequence_to_indices(data_text):
    data_index = []

    for sequence in data_text:
        step_indices = []
        for step in sequence:
            step_indices.append(step_to_index[step])
        data_index.append(step_indices)

    return data_index

In [ ]:
data_indices = sequence_to_indices(data_text)
print(data_indices[0])
len(data_indices)

# Find the maximum and minimum number attack sequence length

In [ ]:
# find max and min number of steps
max = 0
min = 100
with open(sequences_csv, "r") as read_file:
    reader = csv.reader(read_file)
    
    for row in reader:
        length = len(row)
        if length > max:
            max = length
        if length < min:
            min = length

print("Maximum number of steps in the sequences: ", max)
print("Minimum number of steps in the sequences: ", min)

Maximum number of steps in the sequences:  35
Minimum number of steps in the sequences:  4


# Find the unique attack steps

In [ ]:
# find unique steps from the sequences

unique_steps = set()
with open(sequences_csv, "r") as read_file:
    reader = csv.reader(read_file)

    for row in reader:
        for step in row:
            unique_steps.add(step)

print("Total Unique Steps:", len(unique_steps))

Total Unique Steps: 50


In [ ]:
# Save unique steps in a file
unique_steps_filepath = "/content/drive/MyDrive/Thesis/Attack Step Prediction/Implementation/unique_steps.txt"

with open(file_path, "w") as write_file:
    for step in sorted(unique_steps):
        write_file.write(step + "\n")

# Implement step to index and index to step conversion

In [ ]:
def get_conversion_dicts():

    # create dicts for step_to_index conversion and vice-versa
    unique_steps_filepath = "/content/drive/MyDrive/Thesis/Attack Step Prediction/Implementation/unique_steps.txt"

    step_to_index = {}
    index_to_step = {}

    with open(unique_steps_filepath, "r") as read_file:
        lines = read_file.readlines()

        for idx, line in enumerate(lines):
            line = line.strip()     
            idx += 1    # will start indexing from 1
            step_to_index[line] = idx
            index_to_step[idx] = line

    return step_to_index, index_to_step

In [ ]:
step_to_index, index_to_step = get_conversion_dicts()
print(step_to_index['defense_evasion TA0005 Component_Object_Model_Hijacking T1122'])
print(index_to_step[10])

10
defense_evasion TA0005 Component_Object_Model_Hijacking T1122


# Find number of occurences of each attack step
For example, `('discovery TA0007 File_and_Directory_Discovery T1083', 597)` means this tactic has been found in 597 attack sequences out of 800 sequences.

In [ ]:
import collections

sequences_csv = "/content/drive/MyDrive/Thesis/Attack Step Prediction/Implementation/sequences.csv"
unique_steps_filepath = "/content/drive/MyDrive/Thesis/Attack Step Prediction/Implementation/unique_steps.txt"
step_occurence_count = {}

with open(sequences_csv, "r", newline="") as sequence_file:
    sequence_reader = csv.reader(sequence_file)
    sequences = list(sequence_reader)

with open(unique_steps_filepath, "r", newline="") as unique_steps_file:
        unique_steps = unique_steps_file.readlines()
        for step in unique_steps:
            step = step.strip()
            cnt = 0
            for sequence in sequences:
                if step in sequence:
                    cnt += 1
            step_occurence_count[step] = cnt

sorted_occurence_count = sorted(step_occurence_count.items(), key=lambda item: item[1], reverse=True)
for item in sorted_occurence_count:
    print(item)

In [ ]:
step_frequency_count = {}
with open(unique_steps_filepath, "r", newline="") as unique_steps_file:
        unique_steps = unique_steps_file.readlines()
        for step in unique_steps:
            step = step.strip()
            step_frequency_count[step] = 0
cnt = 0
with open(sequences_csv, "r", newline="") as sequence_file:
    sequence_reader = csv.reader(sequence_file)
    for sequence in sequence_reader:
        for step in sequence:
            cnt += 1
            step_frequency_count[step] = step_frequency_count[step] + 1

print("Total attack steps:", cnt)
sorted_frequency_count = sorted(step_frequency_count.items(), key=lambda item: item[1], reverse=True)
for item in sorted_frequency_count:
    print(item)

Total attack steps: 10830
('credential_access TA0006 Credential_Dumping T1003', 1038)
('collection TA0009 Data_Staged T1074', 861)
('discovery TA0007 File_and_Directory_Discovery T1083', 807)
('defense_evasion TA0005 File_Deletion T1107', 788)
('discovery TA0007 Remote_System_Discovery T1018', 741)
('defense_evasion TA0005 Deobfuscate/Decode_Files_or_Information T1140', 621)
('exfiltration TA0010 Exfiltration_Over_Command_and_Control_Channel T1041', 620)
('lateral_movement TA0008 Remote_File_Copy T1105', 474)
('collection TA0009 Input_Capture T1056', 431)
('collection TA0009 Email_Collection T1114', 361)
('defense_evasion TA0005 Indicator_Removal_on_Host T1070', 285)
('credential_access TA0006 Input_Capture T1056', 270)
('discovery TA0007 System_Time_Discovery T1124', 248)
('credential_access TA0006 Credentials_in_Files T1081', 244)
('persistence TA0003 New_Service T1050', 233)
('persistence TA0003 Scheduled_Task T1053', 202)
('credential_access TA0006 Credentials_in_Registry T1214', 1